周内效应——定投篇

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py
import numpy as np

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
asset = 'jqdata.000300_XSHG|1d'
# baselineasset = 'jrj.510310'
# asset = 'jrj.110011'
# baselineasset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-12-31', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# # 买入参数，用全部的钱来买入（也就是复利）
# paramsbuy2 = trdb2py.trading2_pb2.BuyParams(
#     perHandMoney=1/6,
# )

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy2 = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=0.5,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

paramsaip = trdb2py.trading2_pb2.AIPParams(
    money=10000,
    type=trdb2py.trading2_pb2.AIPTT_MONTHDAY,
    day=1,
)

# 止盈参数，120%止盈
paramstakeprofit = trdb2py.trading2_pb2.TakeProfitParams(
    perVolume=1,
#     isOnlyProfit=True,
#     isFinish=True,
)

# 止盈参数，120%止盈
paramstakeprofit2 = trdb2py.trading2_pb2.TakeProfitParams(
    perVolume=1,
#     isOnlyProfit=True,
    isFinish=True,
)

# 止盈参数，120%止盈
paramstakeprofit1 = trdb2py.trading2_pb2.TakeProfitParams(
    perVolume=1,
    isOnlyProfit=True,
#     isFinish=True,
)

# 止盈参数，120%止盈
paramstakeprofit3 = trdb2py.trading2_pb2.TakeProfitParams(
    perVolume=1,
    isOnlyProfit=True,
    isFinish=True,
)

# 卖出参数，全部卖出
paramssell7 = trdb2py.trading2_pb2.SellParams(
#     perVolume=1,
    keepTime=7 * 24 * 60 * 60,
)

lststart = [1, 2, 3, 4, 5]
lsttitle = ['周一', '周二', '周三', '周四', '周五']

In [ ]:
def calcweekday2val2(wday, offday):
    if offday == 1:
        if wday == 5:
            return 3
    if offday == 2:
        if wday >= 4:
            return 4
    if offday == 3:
        if wday >= 3:
            return 5
    if offday == 4:
        if wday >= 2:
            return 6
        
    return offday


def getAIPLastTs(pnl):
    ctrlnums = len(pnl.lstCtrl)

    if ctrlnums <= 0:
        return -1
    
    if pnl.lstCtrl[ctrlnums - 1].type == trdb2py.trading2_pb2.CTRL_SELL:
        return pnl.lstCtrl[ctrlnums - 1].ts
    
    return -1

def getLastResult(pnl) -> dict:
    nums = len(pnl.values)

    if nums <= 0:
        return None
    
    return {'cost': pnl.values[nums - 1].cost, 'value': pnl.values[nums - 1].value}

def getLastCtrl(pnl):
    ctrlnums = len(pnl.lstCtrl)

    if ctrlnums <= 0:
        return None
    
    return pnl.lstCtrl[ctrlnums - 1]


def getPNLValueWithTimestamp(ts, pnl: trdb2py.trading2_pb2.PNLAssetData) -> int:
    for i in range(0, len(pnl.values)):
        if ts == pnl.values[i].ts:
            return i

        if ts < pnl.values[i].ts:
            pnl.values.insert(i, trdb2py.trading2_pb2.PNLDataValue(ts=ts))

            return i

    pnl.values.append(trdb2py.trading2_pb2.PNLDataValue(ts=ts))

    return len(pnl.values) - 1

def mergePNL(lstpnl: list) -> trdb2py.trading2_pb2.PNLAssetData:
    pnl = trdb2py.trading2_pb2.PNLAssetData()

    for vpnl in lstpnl:
        v = vpnl['pnl']

        for cai in range(0, len(v.values)):
            di = getPNLValueWithTimestamp(v.values[cai].ts, pnl)
            pnl.values[di].value += v.values[cai].value
            pnl.values[di].cost += v.values[cai].cost

            if pnl.values[di].cost > 0:
                pnl.values[di].perValue = pnl.values[di].value / \
                    pnl.values[di].cost
            else:
                pnl.values[di].perValue = 1
                
#             pnl.values[di].perValue = 2

    return pnl


def mergePNLEx(pnldest:trdb2py.trading2_pb2.PNLAssetData, pnlsrc:trdb2py.trading2_pb2.PNLAssetData, inmoney):
    for cai in range(0, len(pnlsrc.values)):
        di = getPNLValueWithTimestamp(pnlsrc.values[cai].ts, pnldest)
        pnldest.values[di].value += (pnlsrc.values[cai].value - inmoney)
#         pnl.values[di].cost += pnlsrc.values[cai].cost

        if pnldest.values[di].cost > 0:
            pnldest.values[di].perValue = pnldest.values[di].value / \
                pnldest.values[di].cost
        else:
            pnldest.values[di].perValue = 1
            


def getNextMonthDay1(ts):
    dt = datetime.utcfromtimestamp(ts)
    
    if dt.month == 12:
        return datetime(dt.year + 1, 1, 1).timestamp()
    
    return datetime(dt.year, dt.month + 1, 1).timestamp()


def rmPNLValuesWithTimestamp(ts, pnl: trdb2py.trading2_pb2.PNLAssetData):
    i = getPNLValueWithTimestamp(ts, pnl)
    del pnl.values[i+1:]
    
def getPNLTimestampLowInMonth(pnl: trdb2py.trading2_pb2.PNLAssetData) -> list:
    ts = 0
    dt = None
    lastPerValue = 0
    arr = []

    for i in range(0, len(pnl.values)):
        v = pnl.values[i]

        if ts == 0:
            ts = v.ts
            dt = datetime.utcfromtimestamp(ts)
            lastPerValue = v.perValue
        else:
            cdt = datetime.utcfromtimestamp(v.ts)
            if dt.year == cdt.year and dt.month == cdt.month:
                if lastPerValue > v.perValue:
                    ts = v.ts
                    dt = cdt
                    lastPerValue = v.perValue

                if i == len(pnl.values) - 1:
                    arr.append(ts)
            else:
                arr.append(ts)

                ts = v.ts
                dt = cdt
                lastPerValue = v.perValue

    return arr


def getPNLTimestampHighInMonth(pnl: trdb2py.trading2_pb2.PNLAssetData) -> list:
    ts = 0
    dt = None
    lastPerValue = 0
    arr = []

    for i in range(0, len(pnl.values)):
        v = pnl.values[i]

        if ts == 0:
            ts = v.ts
            dt = datetime.utcfromtimestamp(ts)
            lastPerValue = v.perValue
        else:
            cdt = datetime.utcfromtimestamp(v.ts)
            if dt.year == cdt.year and dt.month == cdt.month:
                if lastPerValue < v.perValue:
                    ts = v.ts
                    dt = cdt
                    lastPerValue = v.perValue

                if i == len(pnl.values) - 1:
                    arr.append(ts)
            else:
                arr.append(ts)

                ts = v.ts
                dt = cdt
                lastPerValue = v.perValue

    return arr

In [ ]:
asset = 'jrj.110011'
# asset = 'jqdata.000036_XSHG|1d'
# asset = 'jqdata.000032_XSHG|1d'
asset = 'jqdata.000300_XSHG|1d'

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

for i in range(1, 2):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[i],
    )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )
    
#     paramsaip = trdb2py.trading2_pb2.AIPParams(
#         money=10000,
#         type=trdb2py.trading2_pb2.AIPTT_WEEKDAY,
#         day=1,
#     )

    s0.buy.extend([buy0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
#     s0.paramsInit.CopyFrom(paramsinit)
    s0.paramsAIP.CopyFrom(paramsaip)
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='每月{}号定投'.format(i),
    ))
 
lstaippnl = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstaippnl + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

我们看到不管是每个月的几号买入，最终其实都差异不大

In [ ]:
dfpnl1b = trdb2py.buildPNLReport(lstaippnl + [pnlBaseline])

dfpnl1b[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility']].sort_values(by='totalReturns', ascending=False)

In [ ]:
lstparams = []
lstpnl = []
cost = 0
value = 0

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
i = 1

while True:
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[1],
    )

#     takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
# #         name='annualizedreturns',
#         name='totalreturnex',        
#         vals=[1.2, 0.1],
#         operators=['>='],
#     )
    takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
#         name='annualizedreturns',
        name='totalreturn',
        vals=[1.2],
        operators=['>='],
    )
    
    takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
        name='timestamp',
        int64Vals=[tsStart + 60 * 60 * 24 * 180],
        operators=['>='],
    )    

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.takeprofit.extend([takeprofit0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
    s0.paramsAIP.CopyFrom(paramsaip)
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='20%止盈{}'.format(i),
    ) 
    pnlt = trdb2py.simTrading(trdb2cfg, p0)
    pnlrt = getLastResult(pnlt['pnl'])
    cost += pnlrt['cost']
    value += pnlrt['value']    
    
    lstpnl.append(pnlt)
    
    cts = getAIPLastTs(pnlt['pnl'])
    if cts < 0:
        break
    
    tsStart = int(getNextMonthDay1(cts))
    i+=1
    
# {'value': value, 'cost': cost, 'per': value / cost}
trdb2py.showPNLs(lstpnl + [pnlBaseline] + lstaippnl, toImg=isStaticImg, width=width, height=height)

In [ ]:
pnldata = mergePNL(lstpnl)
# pnldata = mergePNL([])

trdb2py.showPNLs(lstpnl + [pnlBaseline, {'title': '20%止盈合并', 'pnl': pnldata}, lstaippnl[0]], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []
lstpnl1 = []
cost = 0
value = 0

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
i = 1

while True:
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[1],
    )

#     takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
# #         name='annualizedreturns',
#         name='totalreturnex',        
#         vals=[1.2, 0.1],
#         operators=['>='],
#     )
    takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
#         name='annualizedreturns',
        name='totalreturn',
        vals=[1.2],
        operators=['>='],
    )
    
    takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
        name='timestamp',
        int64Vals=[tsStart + 60 * 60 * 24 * 180],
        operators=['>='],
    )    

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.takeprofit.extend([takeprofit0, takeprofit1])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
    s0.paramsAIP.CopyFrom(paramsaip)
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='20%止盈至少6期{}'.format(i),
    ) 
    pnlt = trdb2py.simTrading(trdb2cfg, p0)
    pnlrt = getLastResult(pnlt['pnl'])
    cost += pnlrt['cost']
    value += pnlrt['value']    
    
    lstpnl1.append(pnlt)
    
    cts = getAIPLastTs(pnlt['pnl'])
    if cts < 0:
        break
    
    tsStart = int(getNextMonthDay1(cts))
    i+=1
    
# {'value': value, 'cost': cost, 'per': value / cost}
trdb2py.showPNLs(lstpnl1 + [pnlBaseline] + lstaippnl, toImg=isStaticImg, width=width, height=height)

In [ ]:
pnldata1 = mergePNL(lstpnl1)
# pnldata = mergePNL([])

trdb2py.showPNLs([pnlBaseline, {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, lstaippnl[0], {'title': '20%止盈合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []
lstpnl2 = []
cost = 0
value = 0

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
i = 1

while True:
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[1],
    )

    takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
#         name='annualizedreturns',
        name='totalreturnex',        
        vals=[1.2, 0.1],
        operators=['>='],
    )
#     takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
#         name='annualizedreturns',
# #         name='totalreturn',
#         vals=[1.2],
#         operators=['>='],
#     )
    
#     takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
#         name='timestamp',
#         int64Vals=[tsStart + 60 * 60 * 24 * 180],
#         operators=['>='],
#     )    

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.takeprofit.extend([takeprofit0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
    s0.paramsAIP.CopyFrom(paramsaip)
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='20%回撤10%止盈{}'.format(i),
    ) 
    pnlt = trdb2py.simTrading(trdb2cfg, p0)
    pnlrt = getLastResult(pnlt['pnl'])
    cost += pnlrt['cost']
    value += pnlrt['value']    
    
    lstpnl2.append(pnlt)
    
    cts = getAIPLastTs(pnlt['pnl'])
    if cts < 0:
        break
    
    tsStart = int(getNextMonthDay1(cts))
    i+=1
    
# {'value': value, 'cost': cost, 'per': value / cost}
trdb2py.showPNLs(lstpnl2 + [pnlBaseline] + lstaippnl, toImg=isStaticImg, width=width, height=height)

In [ ]:
pnldata2 = mergePNL(lstpnl2)
# pnldata = mergePNL([])

trdb2py.showPNLs([pnlBaseline, {'title': '20%回撤10%止盈合并', 'pnl': pnldata2}, {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, lstaippnl[0], {'title': '20%止盈合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []
lstpnl3 = []
cost = 0
value = 0

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
i = 1

while True:
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[1],
    )

#     takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
# #         name='annualizedreturns',
#         name='totalreturnex',        
#         vals=[1.2, 0.1],
#         operators=['>='],
#     )
    takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
#         name='annualizedreturns',
        name='totalreturn',
        vals=[2.0],
        operators=['>='],
    )
    
    takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
        name='timestamp',
        int64Vals=[tsStart + 60 * 60 * 24 * 180],
        operators=['>='],
    )    

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.takeprofit.extend([takeprofit0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
    s0.paramsAIP.CopyFrom(paramsaip)
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='100%止盈{}'.format(i),
    ) 
    pnlt = trdb2py.simTrading(trdb2cfg, p0)
    pnlrt = getLastResult(pnlt['pnl'])
    cost += pnlrt['cost']
    value += pnlrt['value']    
    
    lstpnl3.append(pnlt)
    
    cts = getAIPLastTs(pnlt['pnl'])
    if cts < 0:
        break
    
    tsStart = int(getNextMonthDay1(cts))
    i+=1
    
# {'value': value, 'cost': cost, 'per': value / cost}
trdb2py.showPNLs(lstpnl3 + [pnlBaseline] + lstaippnl, toImg=isStaticImg, width=width, height=height)

In [ ]:
pnldata3 = mergePNL(lstpnl3)
# pnldata = mergePNL([])

trdb2py.showPNLs([pnlBaseline, 
                  {'title': '100%止盈合并', 'pnl': pnldata3}, 
                  {'title': '20%回撤10%止盈合并', 'pnl': pnldata2}, 
                  {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, 
                  lstaippnl[0], 
                  {'title': '20%止盈合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []
lstpnl4 = []
lstpnl41 = []
cost = 0
value = 0

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
i = 1
lastMoney = 0

def deeprunAIP(lst, ts, money, parts, index):
    i = 1
    while True:
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='monthdayex',
            vals=[1],
        )

        takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
            name='totalreturn',        
            vals=[2.0],
            operators=['>='],
        )

        takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
            name='timestamp',
            int64Vals=[tsStart + 60 * 60 * 24 * 180],
            operators=['>='],
        )    
        
        # 初始资金池
        paramsinitt = trdb2py.trading2_pb2.InitParams(
            money=money,
        )

        # 买入参数，用全部的钱来买入（也就是复利）
        paramsbuyt = trdb2py.trading2_pb2.BuyParams(
            perHandMoney=1/parts,
        )        

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0])
        s0.takeprofit.extend([takeprofit0, takeprofit1])
        s0.paramsBuy.CopyFrom(paramsbuyt)
        s0.paramsSell.CopyFrom(paramssell)
        s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
        s0.paramsInit.CopyFrom(paramsinitt)
#         s0.paramsAIP.CopyFrom(paramsaip)
        p0 = trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=ts,
            endTs=tsEnd,
            strategies=[s0],
            title='100%盈利再定投{}-{}'.format(index, i),
        ) 
        pnlt = trdb2py.simTrading(trdb2cfg, p0)
        pnlt['inmoney'] = money
        
        lst.append(pnlt)

        cts = getAIPLastTs(pnlt['pnl'])
        if cts < 0:
            break
            
        lastctrl = getLastCtrl(pnlt['pnl'])
        money = lastctrl.lastMoney
        ts = int(getNextMonthDay1(cts))
        i+=1        
        


while True:        
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[1],
    )

    takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
#         name='annualizedreturns',
        name='totalreturn',        
        vals=[2.0],
        operators=['>='],
    )
    
    takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
        name='timestamp',
        int64Vals=[tsStart + 60 * 60 * 24 * 180],
        operators=['>='],
    )    

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.takeprofit.extend([takeprofit0, takeprofit1])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
    s0.paramsAIP.CopyFrom(paramsaip)
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='100%止盈{}'.format(i),
    ) 
    pnlt = trdb2py.simTrading(trdb2cfg, p0)
    pnlrt = getLastResult(pnlt['pnl'])
    cost += pnlrt['cost']
    value += pnlrt['value']
    
    lstpnl4.append(pnlt)
    
    cts = getAIPLastTs(pnlt['pnl'])
    if cts < 0:
        break
        
    lastctrl = getLastCtrl(pnlt['pnl'])
    lastMoney += lastctrl.lastMoney
    
    tsStart = int(getNextMonthDay1(cts))
    
    deeprunAIP(lstpnl41, tsStart, lastctrl.lastMoney, 6, i)
    
    i+=1
    
# {'value': value, 'cost': cost, 'per': value / cost}
trdb2py.showPNLs(lstpnl4 + lstpnl41 + [pnlBaseline] + lstaippnl, toImg=isStaticImg, width=width, height=height)

In [ ]:
# getLastCtrl(lstpnl[0]['pnl']).ts

# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[0]['pnl']).ts, lstpnl4[0]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[2]['pnl']).ts, lstpnl4[2]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[4]['pnl']).ts, lstpnl4[4]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[6]['pnl']).ts, lstpnl4[6]['pnl'])
pnldata4 = mergePNL(lstpnl4)

for v in lstpnl41:
    mergePNLEx(pnldata4, v['pnl'], v['inmoney'])
# # pnldata = mergePNL([])

# trdb2py.showPNLs(lstpnl + [pnlBaseline, {'title': '合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)
trdb2py.showPNLs([{'title': '100%止盈再定投合并', 'pnl': pnldata4},
                  pnlBaseline, 
                  {'title': '100%止盈合并', 'pnl': pnldata3}, 
                  {'title': '20%回撤10%止盈合并', 'pnl': pnldata2}, 
                  {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, 
                  lstaippnl[0], 
                  {'title': '20%止盈合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []
lstpnl5 = []
lstpnl51 = []
cost = 0
value = 0

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
i = 1
lastMoney = 0

def deeprunAIP(lst, ts, money, parts, index):
    i = 1
    while True:
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='monthdayex',
            vals=[1],
        )

        takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
            name='totalreturn',        
            vals=[1.2],
            operators=['>='],
        )

        takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
            name='timestamp',
            int64Vals=[tsStart + 60 * 60 * 24 * 180],
            operators=['>='],
        )    
        
        # 初始资金池
        paramsinitt = trdb2py.trading2_pb2.InitParams(
            money=money,
        )

        # 买入参数，用全部的钱来买入（也就是复利）
        paramsbuyt = trdb2py.trading2_pb2.BuyParams(
            perHandMoney=1/parts,
        )        

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0])
        s0.takeprofit.extend([takeprofit0])
        s0.paramsBuy.CopyFrom(paramsbuyt)
        s0.paramsSell.CopyFrom(paramssell)
        s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
        s0.paramsInit.CopyFrom(paramsinitt)
#         s0.paramsAIP.CopyFrom(paramsaip)
        p0 = trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=ts,
            endTs=tsEnd,
            strategies=[s0],
            title='20%盈利再定投{}-{}'.format(index, i),
        ) 
        pnlt = trdb2py.simTrading(trdb2cfg, p0)
        pnlt['inmoney'] = money
        
        lst.append(pnlt)

        cts = getAIPLastTs(pnlt['pnl'])
        if cts < 0:
            break
            
        lastctrl = getLastCtrl(pnlt['pnl'])
        money = lastctrl.lastMoney
        ts = int(getNextMonthDay1(cts))
        i+=1        
        


while True:        
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[1],
    )

    takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
#         name='annualizedreturns',
        name='totalreturn',        
        vals=[1.2],
        operators=['>='],
    )
    
    takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
        name='timestamp',
        int64Vals=[tsStart + 60 * 60 * 24 * 180],
        operators=['>='],
    )    

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.takeprofit.extend([takeprofit0, takeprofit1])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
    s0.paramsAIP.CopyFrom(paramsaip)
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='20%止盈{}'.format(i),
    ) 
    pnlt = trdb2py.simTrading(trdb2cfg, p0)
    pnlrt = getLastResult(pnlt['pnl'])
    cost += pnlrt['cost']
    value += pnlrt['value']
    
    lstpnl5.append(pnlt)
    
    cts = getAIPLastTs(pnlt['pnl'])
    if cts < 0:
        break
        
    lastctrl = getLastCtrl(pnlt['pnl'])
    lastMoney += lastctrl.lastMoney
    
    tsStart = int(getNextMonthDay1(cts))
    
    deeprunAIP(lstpnl51, tsStart, lastctrl.lastMoney, 6, i)
    
    i+=1
    
# {'value': value, 'cost': cost, 'per': value / cost}
trdb2py.showPNLs(lstpnl5 + lstpnl51 + [pnlBaseline] + lstaippnl, toImg=isStaticImg, width=width, height=height)

In [ ]:
# getLastCtrl(lstpnl[0]['pnl']).ts

# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[0]['pnl']).ts, lstpnl4[0]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[2]['pnl']).ts, lstpnl4[2]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[4]['pnl']).ts, lstpnl4[4]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[6]['pnl']).ts, lstpnl4[6]['pnl'])
pnldata5 = mergePNL(lstpnl5)

for v in lstpnl51:
    mergePNLEx(pnldata5, v['pnl'], v['inmoney'])
# # pnldata = mergePNL([])

# trdb2py.showPNLs(lstpnl + [pnlBaseline, {'title': '合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)
trdb2py.showPNLs([{'title': '20%止盈再定投合并', 'pnl': pnldata5},
                  {'title': '100%止盈再定投合并', 'pnl': pnldata4},
                  pnlBaseline, 
                  {'title': '100%止盈合并', 'pnl': pnldata3}, 
                  {'title': '20%回撤10%止盈合并', 'pnl': pnldata2}, 
                  {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, 
                  lstaippnl[0], 
                  {'title': '20%止盈合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []
lstpnl6 = []
cost = 0
value = 0

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
i = 1

while True:
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[1],
    )

#     takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
# #         name='annualizedreturns',
#         name='totalreturnex',        
#         vals=[1.2, 0.1],
#         operators=['>='],
#     )
    takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
        name='annualizedreturns',
#         name='totalreturn',
        vals=[1.2],
        operators=['>='],
    )
    
    takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
        name='timestamp',
        int64Vals=[tsStart + 60 * 60 * 24 * 180],
        operators=['>='],
    )    

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.takeprofit.extend([takeprofit0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
    s0.paramsAIP.CopyFrom(paramsaip)
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='年化20%止盈{}'.format(i),
    ) 
    pnlt = trdb2py.simTrading(trdb2cfg, p0)
    pnlrt = getLastResult(pnlt['pnl'])
    cost += pnlrt['cost']
    value += pnlrt['value']    
    
    lstpnl6.append(pnlt)
    
    cts = getAIPLastTs(pnlt['pnl'])
    if cts < 0:
        break
    
    tsStart = int(getNextMonthDay1(cts))
    i+=1
    
# {'value': value, 'cost': cost, 'per': value / cost}
trdb2py.showPNLs(lstpnl6 + [pnlBaseline] + lstaippnl, toImg=isStaticImg, width=width, height=height)

In [ ]:
# getLastCtrl(lstpnl[0]['pnl']).ts

# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[0]['pnl']).ts, lstpnl4[0]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[2]['pnl']).ts, lstpnl4[2]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[4]['pnl']).ts, lstpnl4[4]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[6]['pnl']).ts, lstpnl4[6]['pnl'])
pnldata6 = mergePNL(lstpnl6)

# for v in lstpnl51:
#     mergePNLEx(pnldata5, v['pnl'], v['inmoney'])
# # pnldata = mergePNL([])

# trdb2py.showPNLs(lstpnl + [pnlBaseline, {'title': '合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)
trdb2py.showPNLs([{'title': '年化20%止盈合并', 'pnl': pnldata6},
                  {'title': '20%止盈再定投合并', 'pnl': pnldata5},
                  {'title': '100%止盈再定投合并', 'pnl': pnldata4},
                  pnlBaseline, 
                  {'title': '100%止盈合并', 'pnl': pnldata3}, 
                  {'title': '20%回撤10%止盈合并', 'pnl': pnldata2}, 
                  {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, 
                  lstaippnl[0], 
                  {'title': '20%止盈合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []
lstpnl7 = []
cost = 0
value = 0

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
i = 1

while True:
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[1],
    )

#     takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
# #         name='annualizedreturns',
#         name='totalreturnex',        
#         vals=[1.2, 0.1],
#         operators=['>='],
#     )
    takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
        name='annualizedreturns',
#         name='totalreturn',
        vals=[1.1],
        operators=['>='],
    )
    
    takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
        name='timestamp',
        int64Vals=[tsStart + 60 * 60 * 24 * 180],
        operators=['>='],
    )    

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.takeprofit.extend([takeprofit0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
    s0.paramsAIP.CopyFrom(paramsaip)
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='年化10%止盈{}'.format(i),
    ) 
    pnlt = trdb2py.simTrading(trdb2cfg, p0)
    pnlrt = getLastResult(pnlt['pnl'])
    cost += pnlrt['cost']
    value += pnlrt['value']    
    
    lstpnl7.append(pnlt)
    
    cts = getAIPLastTs(pnlt['pnl'])
    if cts < 0:
        break
    
    tsStart = int(getNextMonthDay1(cts))
    if tsStart >= tsEnd:
        break
    i+=1
    
# {'value': value, 'cost': cost, 'per': value / cost}
trdb2py.showPNLs(lstpnl7 + [pnlBaseline] + lstaippnl, toImg=isStaticImg, width=width, height=height)

In [ ]:
# getLastCtrl(lstpnl[0]['pnl']).ts

# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[0]['pnl']).ts, lstpnl4[0]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[2]['pnl']).ts, lstpnl4[2]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[4]['pnl']).ts, lstpnl4[4]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[6]['pnl']).ts, lstpnl4[6]['pnl'])
pnldata7 = mergePNL(lstpnl7)

# for v in lstpnl51:
#     mergePNLEx(pnldata5, v['pnl'], v['inmoney'])
# # pnldata = mergePNL([])

# trdb2py.showPNLs(lstpnl + [pnlBaseline, {'title': '合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)
trdb2py.showPNLs([{'title': '年化10%止盈合并', 'pnl': pnldata7},
                  {'title': '年化20%止盈合并', 'pnl': pnldata6},
                  {'title': '20%止盈再定投合并', 'pnl': pnldata5},
                  {'title': '100%止盈再定投合并', 'pnl': pnldata4},
                  pnlBaseline, 
                  {'title': '100%止盈合并', 'pnl': pnldata3}, 
                  {'title': '20%回撤10%止盈合并', 'pnl': pnldata2}, 
                  {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, 
                  lstaippnl[0], 
                  {'title': '20%止盈合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []
lstpnl8 = []
lstpnl81 = []
cost = 0
value = 0

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
i = 1
lastMoney = 0

def deeprunAIP(lst, ts, money, parts, index):
    i = 1
    while True:
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='monthdayex',
            vals=[1],
        )

        takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
            name='annualizedreturns',        
            vals=[1.2],
            operators=['>='],
        )

        takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
            name='timestamp',
            int64Vals=[tsStart + 60 * 60 * 24 * 180],
            operators=['>='],
        )    
        
        # 初始资金池
        paramsinitt = trdb2py.trading2_pb2.InitParams(
            money=money,
        )

        # 买入参数，用全部的钱来买入（也就是复利）
        paramsbuyt = trdb2py.trading2_pb2.BuyParams(
            perHandMoney=1/parts,
        )        

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0])
        s0.takeprofit.extend([takeprofit0, takeprofit1])
        s0.paramsBuy.CopyFrom(paramsbuyt)
        s0.paramsSell.CopyFrom(paramssell)
        s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
        s0.paramsInit.CopyFrom(paramsinitt)
#         s0.paramsAIP.CopyFrom(paramsaip)
        p0 = trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=ts,
            endTs=tsEnd,
            strategies=[s0],
            title='年化20%盈利再定投{}-{}'.format(index, i),
        ) 
        pnlt = trdb2py.simTrading(trdb2cfg, p0)
        pnlt['inmoney'] = money
        
        lst.append(pnlt)

        cts = getAIPLastTs(pnlt['pnl'])
        if cts < 0:
            break
            
        lastctrl = getLastCtrl(pnlt['pnl'])
        money = lastctrl.lastMoney
        ts = int(getNextMonthDay1(cts))
        i+=1        
        


while True:        
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[1],
    )

    takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
        name='annualizedreturns',
#         name='totalreturn',        
        vals=[1.2],
        operators=['>='],
    )
    
    takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
        name='timestamp',
        int64Vals=[tsStart + 60 * 60 * 24 * 180],
        operators=['>='],
    )    

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.takeprofit.extend([takeprofit0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsTakeProfit.CopyFrom(paramstakeprofit2)
    s0.paramsAIP.CopyFrom(paramsaip)
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='年化20%止盈{}'.format(i),
    ) 
    pnlt = trdb2py.simTrading(trdb2cfg, p0)
    pnlrt = getLastResult(pnlt['pnl'])
    cost += pnlrt['cost']
    value += pnlrt['value']
    
    lstpnl8.append(pnlt)
    
    cts = getAIPLastTs(pnlt['pnl'])
    if cts < 0:
        break
        
    lastctrl = getLastCtrl(pnlt['pnl'])
    lastMoney += lastctrl.lastMoney
    
    tsStart = int(getNextMonthDay1(cts))
    
    deeprunAIP(lstpnl81, tsStart, lastctrl.lastMoney, 6, i)
    
    i+=1
    
# {'value': value, 'cost': cost, 'per': value / cost}
trdb2py.showPNLs(lstpnl8 + lstpnl81 + [pnlBaseline] + lstaippnl, toImg=isStaticImg, width=width, height=height)

In [ ]:
# getLastCtrl(lstpnl[0]['pnl']).ts

# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[0]['pnl']).ts, lstpnl4[0]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[2]['pnl']).ts, lstpnl4[2]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[4]['pnl']).ts, lstpnl4[4]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[6]['pnl']).ts, lstpnl4[6]['pnl'])
pnldata8 = mergePNL(lstpnl8)

for v in lstpnl81:
    mergePNLEx(pnldata8, v['pnl'], v['inmoney'])
# # pnldata = mergePNL([])

# trdb2py.showPNLs(lstpnl + [pnlBaseline, {'title': '合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)
trdb2py.showPNLs([{'title': '年化20%止盈再定投合并', 'pnl': pnldata8},
                  {'title': '年化10%止盈合并', 'pnl': pnldata7},
                  {'title': '年化20%止盈合并', 'pnl': pnldata6},
                  {'title': '20%止盈再定投合并', 'pnl': pnldata5},
                  {'title': '100%止盈再定投合并', 'pnl': pnldata4},
                  pnlBaseline, 
                  {'title': '100%止盈合并', 'pnl': pnldata3}, 
                  {'title': '20%回撤10%止盈合并', 'pnl': pnldata2}, 
                  {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, 
                  lstaippnl[0], 
                  {'title': '20%止盈合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []
lstpnl9 = []
cost = 0
value = 0

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
i = 1

while True:
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[1],
    )

#     takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
# #         name='annualizedreturns',
#         name='totalreturnex',        
#         vals=[1.2, 0.1],
#         operators=['>='],
#     )
    takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
#         name='annualizedreturns',
        name='totalreturn',
        vals=[1.2],
        operators=['>='],
    )
    
    takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
        name='timestamp',
        int64Vals=[tsStart + 60 * 60 * 24 * 180],
        operators=['>='],
    )    

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.takeprofit.extend([takeprofit0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsTakeProfit.CopyFrom(paramstakeprofit3)
    s0.paramsAIP.CopyFrom(paramsaip)
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='20%部分止盈{}'.format(i),
    ) 
    pnlt = trdb2py.simTrading(trdb2cfg, p0)
    pnlrt = getLastResult(pnlt['pnl'])
    cost += pnlrt['cost']
    value += pnlrt['value']    
    
    lstpnl9.append(pnlt)
    
    cts = getAIPLastTs(pnlt['pnl'])
    if cts < 0:
        break
    
    tsStart = int(getNextMonthDay1(cts))
    i+=1
    
# {'value': value, 'cost': cost, 'per': value / cost}
trdb2py.showPNLs(lstpnl9 + [pnlBaseline] + lstaippnl, toImg=isStaticImg, width=width, height=height)

In [ ]:
# getLastCtrl(lstpnl[0]['pnl']).ts

# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[0]['pnl']).ts, lstpnl4[0]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[2]['pnl']).ts, lstpnl4[2]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[4]['pnl']).ts, lstpnl4[4]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[6]['pnl']).ts, lstpnl4[6]['pnl'])
pnldata9 = mergePNL(lstpnl9)

# for v in lstpnl81:
#     mergePNLEx(pnldata8, v['pnl'], v['inmoney'])
# # pnldata = mergePNL([])

# trdb2py.showPNLs(lstpnl + [pnlBaseline, {'title': '合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)
trdb2py.showPNLs([{'title': '20%部分止盈合并', 'pnl': pnldata9},
#                   {'title': '年化20%止盈再定投合并', 'pnl': pnldata8},
#                   {'title': '年化10%止盈合并', 'pnl': pnldata7},
#                   {'title': '年化20%止盈合并', 'pnl': pnldata6},
#                   {'title': '20%止盈再定投合并', 'pnl': pnldata5},
#                   {'title': '100%止盈再定投合并', 'pnl': pnldata4},
                  pnlBaseline, 
#                   {'title': '100%止盈合并', 'pnl': pnldata3}, 
                  {'title': '20%回撤10%止盈合并', 'pnl': pnldata2}, 
                  {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, 
                  lstaippnl[0], 
                  {'title': '20%止盈合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []
lstpnl10 = []
lstpnl101 = []
cost = 0
value = 0

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
i = 1
lastMoney = 0

def deeprunAIP(lst, ts, money, parts, index):
    i = 1
    while True:
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='monthdayex',
            vals=[1],
        )

        takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
            name='totalreturn',        
            vals=[1.2],
            operators=['>='],
        )

        takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
            name='timestamp',
            int64Vals=[tsStart + 60 * 60 * 24 * 180],
            operators=['>='],
        )    
        
        # 初始资金池
        paramsinitt = trdb2py.trading2_pb2.InitParams(
            money=money,
        )

        # 买入参数，用全部的钱来买入（也就是复利）
        paramsbuyt = trdb2py.trading2_pb2.BuyParams(
            perHandMoney=1/parts,
        )        

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0])
        s0.takeprofit.extend([takeprofit0])
        s0.paramsBuy.CopyFrom(paramsbuyt)
        s0.paramsSell.CopyFrom(paramssell)
        s0.paramsTakeProfit.CopyFrom(paramstakeprofit3)
        s0.paramsInit.CopyFrom(paramsinitt)
#         s0.paramsAIP.CopyFrom(paramsaip)
        p0 = trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=ts,
            endTs=tsEnd,
            strategies=[s0],
            title='20%部分止盈再定投{}-{}'.format(index, i),
        ) 
        pnlt = trdb2py.simTrading(trdb2cfg, p0)
        pnlt['inmoney'] = money
        
        lst.append(pnlt)

        cts = getAIPLastTs(pnlt['pnl'])
        if cts < 0:
            break
            
        lastctrl = getLastCtrl(pnlt['pnl'])
        money = lastctrl.lastMoney
        ts = int(getNextMonthDay1(cts))
        i+=1        
        


while True:        
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='monthdayex',
        vals=[1],
    )

    takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
#         name='annualizedreturns',
        name='totalreturn',        
        vals=[1.2],
        operators=['>='],
    )
    
    takeprofit1 = trdb2py.trading2_pb2.CtrlCondition(
        name='timestamp',
        int64Vals=[tsStart + 60 * 60 * 24 * 180],
        operators=['>='],
    )    

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0])
    s0.takeprofit.extend([takeprofit0, takeprofit1])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell)
    s0.paramsTakeProfit.CopyFrom(paramstakeprofit3)
    s0.paramsAIP.CopyFrom(paramsaip)
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='20%部分止盈{}'.format(i),
    ) 
    pnlt = trdb2py.simTrading(trdb2cfg, p0)
    pnlrt = getLastResult(pnlt['pnl'])
    cost += pnlrt['cost']
    value += pnlrt['value']
    
    lstpnl10.append(pnlt)
    
    cts = getAIPLastTs(pnlt['pnl'])
    if cts < 0:
        break
        
    lastctrl = getLastCtrl(pnlt['pnl'])
    lastMoney += lastctrl.lastMoney
    
    tsStart = int(getNextMonthDay1(cts))
    
    deeprunAIP(lstpnl101, tsStart, lastctrl.lastMoney, 6, i)
    
    i+=1
    
# {'value': value, 'cost': cost, 'per': value / cost}
trdb2py.showPNLs(lstpnl10 + lstpnl101 + [pnlBaseline] + lstaippnl, toImg=isStaticImg, width=width, height=height)

In [ ]:
# getLastCtrl(lstpnl[0]['pnl']).ts

# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[0]['pnl']).ts, lstpnl4[0]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[2]['pnl']).ts, lstpnl4[2]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[4]['pnl']).ts, lstpnl4[4]['pnl'])
# rmPNLValuesWithTimestamp(getLastCtrl(lstpnl4[6]['pnl']).ts, lstpnl4[6]['pnl'])
pnldata10 = mergePNL(lstpnl10)

for v in lstpnl101:
    mergePNLEx(pnldata10, v['pnl'], v['inmoney'])
# # pnldata = mergePNL([])

# trdb2py.showPNLs(lstpnl + [pnlBaseline, {'title': '合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)
trdb2py.showPNLs([{'title': '20%部分止盈再定投合并', 'pnl': pnldata10},
                  {'title': '20%部分止盈合并', 'pnl': pnldata9},
#                   {'title': '年化20%止盈再定投合并', 'pnl': pnldata8},
#                   {'title': '年化10%止盈合并', 'pnl': pnldata7},
#                   {'title': '年化20%止盈合并', 'pnl': pnldata6},
                  {'title': '20%止盈再定投合并', 'pnl': pnldata5},
                  {'title': '100%止盈再定投合并', 'pnl': pnldata4},
                  pnlBaseline, 
                  {'title': '100%止盈合并', 'pnl': pnldata3}, 
                  {'title': '20%回撤10%止盈合并', 'pnl': pnldata2}, 
                  {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, 
                  lstaippnl[0], 
                  {'title': '20%止盈合并', 'pnl': pnldata}], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstallpnl = [{'title': '20%部分止盈合并', 'pnl': pnldata9},
             {'title': '20%部分止盈再定投合并', 'pnl': pnldata10},
             {'title': '年化20%止盈再定投合并', 'pnl': pnldata8},
                  {'title': '年化10%止盈合并', 'pnl': pnldata7},
                  {'title': '年化20%止盈合并', 'pnl': pnldata6},
                  {'title': '20%止盈再定投合并', 'pnl': pnldata5},
                  {'title': '100%止盈再定投合并', 'pnl': pnldata4},
                  pnlBaseline, 
                  {'title': '100%止盈合并', 'pnl': pnldata3}, 
                  {'title': '20%回撤10%止盈合并', 'pnl': pnldata2}, 
                  {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, 
                  lstaippnl[0], 
                  {'title': '20%止盈合并', 'pnl': pnldata}]

for v in lstallpnl:
    trdb2py.rebuildPNL(v['pnl'])
    
dflstallpnl = trdb2py.buildPNLReport(lstallpnl)

dflstallpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility']].sort_values(by='totalReturns', ascending=False)

In [ ]:
{'title': '20%部分止盈合并', 'pnl': pnldata9},
             {'title': '20%部分止盈再定投合并', 'pnl': pnldata10},
             {'title': '年化20%止盈再定投合并', 'pnl': pnldata8},
                  {'title': '年化10%止盈合并', 'pnl': pnldata7},
                  {'title': '年化20%止盈合并', 'pnl': pnldata6},
                  {'title': '20%止盈再定投合并', 'pnl': pnldata5},
                  {'title': '100%止盈再定投合并', 'pnl': pnldata4},
                  pnlBaseline, 
                  {'title': '100%止盈合并', 'pnl': pnldata3}, 
                  {'title': '20%回撤10%止盈合并', 'pnl': pnldata2}, 
                  {'title': '20%止盈至少6期合并', 'pnl': pnldata1}, 
                  lstaippnl[0], 
                  {'title': '20%止盈合并', 'pnl': pnldata}

In [ ]:
lstparams = []

tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
# tsStart = int(trdb2py.str2timestamp('2014-11-01', '%Y-%m-%d'))
# tsStart = int(trdb2py.str2timestamp('2015-01-01', '%Y-%m-%d'))
# tsStart = int(trdb2py.str2timestamp('2015-04-01', '%Y-%m-%d'))
# tsStart = int(trdb2py.str2timestamp('2017-12-01', '%Y-%m-%d'))


buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='monthdayex',
    vals=[1],
)

takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
    name='totalreturn',
    vals=[1.2],
    operators=['>='],
)

s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),
)

# paramsaip = trdb2py.trading2_pb2.AIPParams(
#     money=10000,
#     type=trdb2py.trading2_pb2.AIPTT_WEEKDAY,
#     day=1,
# )

s0.buy.extend([buy0])
s0.takeprofit.extend([takeprofit0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsSell.CopyFrom(paramssell)
s0.paramsTakeProfit.CopyFrom(paramstakeprofit)
s0.paramsAIP.CopyFrom(paramsaip)
lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='{}定投止盈'.format(1),
))
 
lstaippnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

# lstaippnl1[0]['pnl'].lstCtrl
# getPNLLastTs(lstaippnl1[0]['pnl'])

In [ ]:
trdb2py.showPNLs(lstaippnl1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))
# tsStart = int(trdb2py.str2timestamp('2014-11-01', '%Y-%m-%d'))
# tsStart = int(trdb2py.str2timestamp('2015-01-01', '%Y-%m-%d'))
# tsStart = int(trdb2py.str2timestamp('2015-04-01', '%Y-%m-%d'))
# tsStart = int(trdb2py.str2timestamp('2017-12-01', '%Y-%m-%d'))


buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='monthdayex',
    vals=[1],
)

takeprofit0 = trdb2py.trading2_pb2.CtrlCondition(
    name='totalreturn',
    vals=[1.2],
    operators=['>='],
)

s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),
)

# paramsaip = trdb2py.trading2_pb2.AIPParams(
#     money=10000,
#     type=trdb2py.trading2_pb2.AIPTT_WEEKDAY,
#     day=1,
# )

s0.buy.extend([buy0])
s0.takeprofit.extend([takeprofit0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsSell.CopyFrom(paramssell)
s0.paramsTakeProfit.CopyFrom(paramstakeprofit1)
s0.paramsAIP.CopyFrom(paramsaip)
lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='{}定投止盈2'.format(1),
))
 
lstaippnl2 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstaippnl1 + lstaippnl2 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)